# TTSVD and HOSVD for 3d tensors as proof of concept

Consider 3D array as tensor:
$$A(i, j,k) \in \mathbb{R}^{N, N, N}.$$
One simply needs $N^3$ memory cells for its storage. Sometimes the data has structure allowing to parametrize it within some format requiring less parameters.

For example, any 3D array can be represented in one of three (maybe more) tensor formats:

#### Canonical format

Canonical-polyadic format (CP format):
$$
A(i,j,k) = \sum_{\alpha=1}^{R} u_{\alpha}(i) v_{\alpha}(j) w_{\alpha}(k),
$$
giving us a representation of $N \times N \times N$ array with $3 N R$ parameters. It is obvious that for $R \ll N$ we get a great compression of the initial data. 

![Canonical decomposition as picture](CP_picture.png)

E.g. for the function-generated array $A(i,j,k) = \sin(i+j+k)$ one can open the brackets for the $\sin$ function and get only 6 terms. The trouble with CP-format is that computation of the canonical tensor rank $R = trank A$ (minimal possible number of terms for elementwise equality between the decomposed and initial tensors) is a really tough problem. Moreover, if one has some initial guess or target on value of terms in the decomposition then reconstruction of its terms $u_{\alpha}(i) v_{\alpha}(j) w_{\alpha}(k)$ is also a hard problem. The popular alternating least squares algorithm works quite well but may suffer from stagnation of the convergence and there is no theoretical justification of its global convergence in terms of real accuracy. Thus, people really needed some other tensor formats allowing to obtain SVD-related robust procedues.

Anyways it is important to see that any tensor $\bf{A}$ is represantable within CP-format as
$$
\textbf{A} = \sum_{A(i,j,k) \neq 0} e_{j} \otimes e_{j} \otimes e_{k} \cdot A(i,j,k).
$$
It means that we can decompose the tensor simply as a sum of indicators among its all nonzero elements. Such a representation is useless but demonstrates easily existence of the CP-decomposition for arbitrary tensor. Note that by $\otimes$ we note a classical Kronecker product operation and vectors $e_{i}$ correspond to canonical vectors with only one element number at position $i$ equal to unity.

Moreover, such an example shows that the task of finding such a representation with modest number of terms is interesring and may lead to really many benefits. E.g. CP-representation of the $\sin(a i+ b j + c k)$ function for any real values of parameters $a$, $b$, $c$ leads to efficient numerical procedures for functions of such class. 

In this snippet we do not concentrate on this decomposition, even though it is really important as it and also the Tucker and the Tensor Train formats.

#### Literature

1. Vervliet N., Debals O., De Lathauwer L. (2016, November). Tensorlab 3.0—Numerical optimization strategies for large-scale constrained and coupled matrix/tensor factorization. In 2016 50th Asilomar Conference on Signals, Systems and Computers (pp. 1733-1738). IEEE.
2. Kolda, T. G., Bader, B. W. (2009). Tensor decompositions and applications. SIAM review, 51(3), 455-500.
3. Cichocki A., Lee N., Oseledets I., Phan A. H., Zhao Q., Mandic D. P. (2016). Tensor networks for dimensionality reduction and large-scale optimization: Part 1 low-rank tensor decompositions. Foundations and Trends® in Machine Learning, 9(4-5), 249-429.

#### Tucker format
Tucker format gives us the following representation for tensor:
$$
A(i,j,k) = \sum_{\alpha_1=1}^{r_1} \sum_{\alpha_2=1}^{r_2} \sum_{\alpha_3=1}^{r_3} G(\alpha_1, \alpha_2, \alpha_3) U_1(i, \alpha_1) U_2(j, \alpha_2) U_3(k, \alpha_3).
$$
In this case we have to discuss the definition of the multilinear Tucker ranks $(r_1,r_2,r_3)$ which exist and correspond to minimal values $r_1,r_2,r_3$ allowing to obtain the elementwise equality between the decomposition and initial tensor $A$. 

In this case we have to store the so-called Tucker core $G(\alpha_1, \alpha_2, \alpha_3) \in \mathbb{R}^{r_1, r_2,r_3}$ and Tucker factors $U_1(i,\alpha_1) \in \mathbb{R}^{N \times r_1}$, $U_2(i,\alpha_2) \in \mathbb{R}^{N \times r_2}$, $U_3(i,\alpha_3) \in \mathbb{R}^{N \times r_3}$. Thus, one needs to store $r_1 r_2 r_3 + N(r_1 + r_2 + r_3)$ elements instead of initial $N^3$. 

Additional storage of Tucker kernel within $r_1 r_2 r_3$ memory cells is a drawback of this format in comparison with CP-format. But a great advantage of the Tucker decomposition is a higher order SVD (HOSVD) and equivalent sequentially truncated HOSVD (st-HOSVD) procedures. For both procedures the complexity is $O(N^4)$ but the st-HOSVD has a 2-3 times better constant.

The minimal values of Tucker ranks are related to ranks of the special unfolding matrices which are formed as reshaped matrices from the initial tensor $A(i,j,k)$:
$$A_1 = A(i, \overline{j,k})~ \texttt{in Python: reshape(A, [N, N*N]), then SVD requires}~ O(N^4)$$
$$A_2 = A(j, \overline{i,k})~ \texttt{in Python: transpose(A, [1,0,2]), reshape(A, [N, N*N]),  then SVD requires}~ O(N^4)$$
$$A_3 = A(k, \overline{i,j})~ \texttt{in Python: transpose(A, [2,0,1]), reshape(A, [N, N*N]), then SVD requires}~ O(N^4)$$

The ranks of these three unfolding matrices are exactly the minimal Tucker ranks $(r_1,r_2,r_3)$ 
. Assume $A_i = U_i \Sigma_i V_i$, then 
$$
G(\alpha_1, \alpha_2, \alpha_3) = U_1^{\top}U_2^{\top}U_3^{\top} A.
$$
This is the classical HOSVD algorithm.

The st-HOSVD comes as sequential rank reduction through the skeleton decompositions (e.g. SVD) and $\texttt{reshape}$, $\texttt{transpose}$ operations as following:
$$
A(i,j,k) = A(i,\overline{j,k}) = \sum_{\alpha_1}^{r_1} U_1(i_1, \alpha_1) \tilde{G}_{23}(\alpha_1, \overline{j,k}) = \sum_{\alpha_1}^{r_1} U_1(i_1, \alpha_1) \tilde{G}_{23}(\alpha_1, \overline{j,k}) = 
$$
$$
\sum_{\alpha_1}^{r_1} U_1(i_1, \alpha_1) \tilde{G}_{23}(j, \overline{\alpha_1,k}) = \sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} U_1(i, \alpha_1) U_2(j, \alpha_2)\tilde{G}_{3}(\alpha_2, \overline{\alpha_1,k}) = 
$$
$$
\sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} U_1(i, \alpha_1) U_2(j, \alpha_2)\tilde{G}_{3}(k, \overline{\alpha_1,\alpha_2}) = \sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} \sum_{\alpha_3}^{r_3} U_1(i, \alpha_1) U_2(j, \alpha_2) U_3(k, \alpha_3) \tilde{G}(\alpha_3, \overline{\alpha_1,\alpha_2})
$$
Thus, after the final reshape and transpose we get the very same Tucker decomposition
$$
\sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} \sum_{\alpha_3}^{r_3} U_1(i, \alpha_1) U_2(j, \alpha_2) U_3(k, \alpha_3) G(\alpha_1, \alpha_2,\alpha_3).
$$
It means, that we had to get the skeleton decomspositions for matrices:

1. $A(i, \overline{j,k})$ -- it takes $O(N^2 \cdot N^2) = O(N^4)$ operations.

2. $\tilde{G}_{23}(j, \overline{\alpha_1,k})$  -- it takes $O(N^2 \cdot N R) = O(N^3 R)$ operations.

3. $\tilde{G}_{3}(k, \overline{\alpha_1,\alpha_2})$ -- it requires $O(\min(N^2 R^2, N R^4))$ operations. 

Total complexity in this case is $O(N^4 + N^3 R + \min(N^2 R^2, N R^4))$ instead of original $O(3 N^4)$ for the classical HOSVD.

Really great fact, is following: use of the truncated SVD in these algorithms allows to obtain the approximate Tucker decomposition with all necessary theorems about it. The practical theorem about the approximate HOSVD is following
$$||A - B_t||_F \leq \sqrt{d}||A - B_{*}||_F,$$
where $B_t$ -- is the result of either the HOSVD or the st-HOSVD (which is mathematically equivalent to the HOSVD) and the $B_{*}$ is the best approximation of the same fixed multilinear rank.

In case of the higher dimensions the HOSVD can be represented with following picture:

![<Tucker tree for 5 dimensional array>](Tucker_tree.png)

And the final representation of the d-dimensional tensor in Tucker format is given by equation:
$$
A(i_1,i_2, \ldots, i_d) = \sum_{\alpha_1=1}^{r_1} \sum_{\alpha_2=1}^{r_2} \ldots \sum_{\alpha_d=1}^{r_d} G(\alpha_1, \alpha_2, \ldots \alpha_d) U_1(i_1, \alpha_1) U_2(i_2, \alpha_2) \ldots U_d(i_d, \alpha_d).
$$
One may easily observe that storage of the Tucker core $G(\alpha_1, \alpha_2, \ldots \alpha_d)$ requires $r_1 \cdot r_2 \cdot \ldots r_d$ memory cells. In case of $r_i = R$ it is obvious $R^{d}$ memory cells and becomes useless for the higher dimensionality $d$.
 
#### Literature

1. Badeau R., Boyer R. (2008). Fast multilinear singular value decomposition for structured tensors. SIAM Journal on Matrix Analysis and Applications, 30(3), 1008-1021.
2. In Russian: Tyrtyshnikov E. E., Matrices, tensors, computations.

#### Tensor-train format

In case three-dimensional arrays the tensor-train format corresponds to the following representation of array:

$$
A(i,j,k) = \sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} G_1(i, \alpha_1) G_2(\alpha_1, j, \alpha_2) G_3(\alpha_2, k).
$$

It is somehow related to the "not fully finalized" Tucker desomposition and requires $N r_1 r_2 + N r_1 + N r_2$ memory cells and does not sees to be a good idea for the compression. Its nice properties become significant for the tensors having higher dimensionality because it does not have an exponential increase of storage requirements in contrast to Tucker decomposition. 

However, it also can be computed with use of SVD-based procedure which is known as TTSVD algorithm which looks as following in case of 3D-arrays:

$$
A(i,\overline{i,k}) = \sum_{\alpha_1=1}^{r_1} G_1(i, \alpha_1) G_{2,3}(\alpha_1, \overline{j,k}) =
$$
$$
\sum_{\alpha_1=1}^{r_1} G_1(i, \alpha_1) G_{2,3}(\overline{\alpha_1, j}, k) =
\sum_{\alpha_1=1}^{r_1} \sum_{\alpha_2=1}^{r_2} G_1(i, \alpha_1) G_{2}(\alpha_1, j, \alpha_2) G_3(\alpha_2, k).
$$
It has the obvious complexity $O(N^4)$. The minimal TT-ranks correspond to the ranks of the bit different unfolding matrices:
$$A_1 = A(i, \overline{j,k})$$
and 
$$A_2 = A(\overline{i,j}, k).$$

Fortunately, approximate truncation of the TTSVD has the same quasioptimal propreties as the truncation of the HOSVD:
$$||A - B_t||_F \leq \sqrt{d}||A - B_{*}||_F,$$
where $B_t$ -- is the result of the TTSVD and the $B_{*}$ is the best approximation of the same fixed TT-ranks.


![<TT-SVD tree for 5 dimensional array>](TT_tree.png)

Final d-dimensional Tensor Train corresponds to the representation
$$
A(i_1,i_2,\ldots, i_d) = \sum_{\alpha_1}^{r_1} \sum_{\alpha_2}^{r_2} \ldots  \sum_{\alpha_{d-1}}^{r_{d-1}} G_1(i_1, \alpha_1) G_2(\alpha_1, i_2, \alpha_2) \ldots G_{d-1}(\alpha_{d-1}, i_d).
$$
Such a representation exists for any d-dimensional array and known in literare as famous matrix-product state because the element of tensor $A(i_1,i_2,\ldots, i_d)$ can be evaluated within $(d-2)$ matrix-by-vector product operations

![Example for TT as matrix product state](TT.png)

#### Literature

1. Oseledets I. V., Tyrtyshnikov E. E. (2009). Breaking the curse of dimensionality, or how to use SVD in many dimensions. SIAM Journal on Scientific Computing, 31(5), 3744-3759.
2. Zhang C., Jeckelmann E., White S. R. (1998). Density matrix approach to local Hilbert space reduction. Physical review letters, 80(12), 2661.

#### Exercises:
1. Prove that TTSVD for three-dimensional arrays really has complexity $O(N^4)$.
2. Formulate the randomized versions for HOSVD, st-HOSVD, TTSVD, what is the complexity for each of them?
3. Prove that Tucker ranks for $\sin(i+j+k)$ are equal to 2.
4. Prove that TT-ranks for $\sin(i+j+k)$ and $i+j+k$ are equal to 2.
5. Implement the HOSVD and TTSVD for the higher dimensions.
6. Implement the low-rank alternating projections for nonnegative tensor decompositions basing on paper: Sultonov A., Matveev S., Budzinskiy S. (2022). Low-rank nonnegative tensor approximation via alternating projections and sketching. arXiv preprint arXiv:2209.02060.
7. Prove that element of a tensor can be evaluated within $O(d R^2)$ operations in TT-format.
8. Prove that $A + B$ and $A * B$ can be expressed economically in TT-format. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.extmath import randomized_svd as rsvd
import time

/home/sergey/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def HOSVD_3d(A, r1, r2, r3):
    # Construct Tucker decomposition of 3D tensor via HOSVD with ranks [r1, r2, r3]
    N1,N2,N3 = A.shape
    #print(A.shape)
    A1=A.reshape(N1, N2*N3)
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V1 = U[:,:r1] 
    V1 = V1.T
    #print("V1 shape", V1.shape, np.linalg.norm(V1, ord=2))
    A1 = np.diag(s[:r1]) @ V[:r1,:] # r1 x N2*N3
    A1 = A1.reshape(r1, N2, N3)
    A1 = A1.transpose([0,2,1])
    A1 = A1.reshape(r1 * N3, N2)# r1 * N3 x N2
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V2 =  V[:r2,:] # r2 x N2
    #print("V2 shape", V2.shape, np.linalg.norm(V2, ord=2))
    U2 = U[:,:r2] @ np.diag(s[:r2]) # r1*N3 x r2
    U2 = U2.reshape(r1, N3, r2) # r1 x N3 x r2
    U2 = U2.transpose([0,2,1]) # r1 x r2 x N3
    U2 = U2.reshape(r1 * r2, N3) # r1*r2 x N3
    U, s, V = np.linalg.svd(U2, full_matrices=False)
    #print(s)
    V3 = V[:r3,:] # U3(r3,j)
    #V3 = V3.T
    #print("V3 shape", V3.shape, np.linalg.norm(V3, ord=2))
    G = U[:,:r3] @ np.diag(s[:r3])# r1 * r2 x r3
    #print("main core shape", G.shape)
    G = G.reshape(r1, r2, r3) # main core
    #G = G.transpose([0,2,1])
    #print(G.shape)
    return G, V1, V2, V3

In [3]:
def restore_tensor_tuck(G, V1, V2, V3):
    # restore the full tensor from Tucker decomposition
    r1, r2, r3 = G.shape
    G = G.reshape(r1 * r2, r3) #r1*r2 x r3
    A1 = G.dot(V3) # r1*r2 x N3
    A1 = A1.reshape(r1, r2, V3.shape[1])# r1 x r2 x N3
    A1 = A1.transpose([0,2,1])# 
    A1 = A1.reshape(r1*V3.shape[1], r2)# r1*N3 x r2
    A1 = A1.dot(V2) # r1*N3 x N2
    A1 = A1.reshape(r1, V3.shape[1], V2.shape[1])#r1 x N3 x N2
    A1 = A1.transpose([1,2,0]) # N3 x N2 x r1
    A1 = A1.reshape(V3.shape[1], V2.shape[1], r1) # N3 x N2 x r1
    A1 = A1.dot(V1) # N3 x N2 x N1
    A1 = A1.transpose([2,1,0]) # N1 x N2 x N3
    return A1

In [4]:
def restore_tensor(G1, G2, G3):
    # restore the full tensor from TT
    N1 = G1.shape[0]
    #print("G1 shape", G1.shape)
    N2 = G2.shape[1]
    #print("G2 shape", G2.shape)
    N3 = G3.shape[1]
    #print("G3 shape", G3.shape)
    B = (G2 @ G3).reshape(G1.shape[1], N2*N3)
    B = G1 @ B
    B = B.reshape(N1, N2, N3)
    #print("B shape ", B.shape)
    #A1 = np.zeros([N1, N2, N3])
    #print("A1 shape", A1.shape)
    #for i in range(N1):
    #    for j in range(N2):
    #        for k in range(N3):
    #            A1[i,j,k] = G1[i,:] @ (G2[:,j,:] @ G3[:,k])
    return B

In [5]:
def compute_err_np_tuck(A, G, V1, V2, V3):
    # restore numpy array from Tucker decomposition and check error in Frobenius nrm
    # || A - A_r||_F , where A_r given via Tucker decomposition 
    err = 0
    r1, r2, r3 = G.shape
    G = G.reshape(r1 * r2, r3) #r1*r2 x r3
    A1 = G.dot(V3) # r1*r2 x N3
    A1 = A1.reshape(r1, r2, V3.shape[1])# r1 x r2 x N3
    A1 = A1.transpose([0,2,1])# 
    A1 = A1.reshape(r1*V3.shape[1], r2)# r1*N3 x r2
    A1 = A1.dot(V2) # r1*N3 x N2
    A1 = A1.reshape(r1, V3.shape[1], V2.shape[1])#r1 x N3 x N2
    A1 = A1.transpose([1,2,0]) # N3 x N2 x r1
    A1 = A1.reshape(V3.shape[1], V2.shape[1], r1) # N3 x N2 x r1
    A1 = A1.dot(V1) # N3 x N2 x N1
    A1 = A1.transpose([2,1,0]) # N1 x N2 x N3
    err = np.linalg.norm(A - A1) #error norm
    print('error in frob nrm ', err)
    return err

In [6]:
def TTSVD_3d(A, r1, r2):
    N1,N2,N3 = A.shape
    #print(A.shape)
    A1=A.reshape(N1, N2*N3)
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    G1 = U[:,:r1] #N1 x r1
    V1 = np.diag(s[:r1]) @ V[:r1, :] # r1 x N2*N3
    #print("V1 shape", V1.shape)
    V1 = V1.reshape(r1 * N2, N3)
    U, s, V = np.linalg.svd(V1, full_matrices=False)
    G2 = U[:,:r2]  # r1*N2 x r2
    G2 = G2.reshape(r1, N2, r2)# r1 x N2 x r2
    G3 = np.diag(s[:r2]) @V[:r2,:] # r2 x N3
    #print("G1 shape", G1.shape)
    #print("G2 shape", G2.shape)
    #print("G3 shape", G3.shape)
    return G1, G2, G3

In [7]:
N = 32
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(2.*i+3.*j+4.*k)

G1, G2, G3 = TTSVD_3d(A,10,10)
A1 = restore_tensor(G1, G2, G3)
err = np.linalg.norm(A - A1)
print("error = ", err)

print('classical')
print('===========================================')
G, V1, V2, V3 = HOSVD_3d(A,2,2,2)
errnp = compute_err_np_tuck(A, G, V1,V2,V3)
print("err tuck", err)


error =  6.847694317866259e-13
classical
error in frob nrm  2.1377588870253997e-13
err tuck 6.847694317866259e-13


#### Progamming exercises:
1. Implement HOSVD and TTSVD with use of accuracy criteris for the choice of ranks.
2. Implement the randomized versions of HOSVD and TTSVD